# Importações


In [135]:
import geopandas
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import random

# Configurações iniciais


In [136]:
load_dotenv()

True

# Leitura dos dados


In [137]:
inventory: pd.DataFrame = pd.read_csv(
    os.environ.get("INVENTORY_FILE_PATH"),
    encoding="ISO-8859-1",
)

In [138]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313


In [139]:
inventory_plot_location: geopandas.GeoDataFrame = geopandas.read_file(
    os.environ.get("INVENTORY_PLOT_LOCATION_FILE_PATH"),
)

In [140]:
inventory_plot_location.head()

,plot_ID,geometry
0,P20,"POLYGON ((840040.725 9674392.229, 840082.852 9..."
1,P02,"POLYGON ((840077.054 9674170.361, 840124.283 9..."
2,P07,"POLYGON ((840331.289 9673499.875, 840372.258 9..."
3,P09,"POLYGON ((840301.102 9673285.583, 840340.905 9..."
4,P08,"POLYGON ((839205.643 9673316.074, 839256.495 9..."


In [141]:
wood_density_dataframe = pd.read_excel(
    os.environ.get("WOOD_DENSITY_FILE_PATH"),
    sheet_name="Data",
)

In [142]:
wood_density_dataframe.head()

,Number,Family,Binomial,"Wood density (g/cm^3), oven dry mass/fresh volume",Region,Reference Number
0,1,Fabaceae,Abarema jupunba,0.78000,South America (tropical),114
1,2,Fabaceae,Abarema jupunba,0.66000,South America (tropical),198
2,3,Fabaceae,Abarema jupunba,0.55104,South America (tropical),52
3,4,Fabaceae,Abarema jupunba,0.53382,South America (tropical),65
4,5,Fabaceae,Abarema jupunba,0.55104,South America (tropical),189


# Exploração dos dados


In [143]:
inventory["scientific.name"].unique(), len(inventory["scientific.name"].unique())

(array(['Licania guianensis', 'Sacoglottis guianensis', 'Pourouma minor',
        'Pouteria gongrijpii', 'Peltogyne lecointei', nan,
        'Protium paniculatum', 'Qualea paraensis', 'Mouriri collocarpa',
        'Eschweilera ovata', 'Pouteria sp.', 'Ormosia grossa',
        'Ocotea cernua', 'Eperua bijuga', 'Brosimum rubescens',
        'Oenocarpus bataua', 'Hevea brasiliensis', 'Inga thibaudiana',
        'Eriotheca globosa', 'Iryanthera paraensis', 'Dipteryx odorata',
        'Sterculia pruriens', 'Miconia poeppigii', 'Sloanea guianensis',
        'Poeppigia procera', 'Xylopia sp.', 'Neea oppositifolia',
        'Geissospermum sericeum', 'Goupia glabra', 'Eschweilera micrantha',
        'Inga alba', 'Diospyros artanthifolia', 'Eschweilera coriacea',
        'Micrandra minor', 'Trichilia micrantha',
        'Tetragastris panamensis', 'Ormosia coccinea', 'Guarea guidonia',
        'Couepia robusta', 'Pouteria guianensis', 'Ocotea canaliculata',
        'Licania octandra', 'Guatteria 

In [144]:
inventory["family.name"].unique(), len(inventory["family.name"].unique())

(array(['Chrysobalanaceae', 'Humiriaceae', 'Urticaceae', 'Sapotaceae',
        'Fabaceae', nan, 'Burseraceae', 'Vochysiaceae', 'Melastomataceae',
        'Lecythidaceae', 'Lauraceae', 'Moraceae', 'Arecaceae',
        'Euphorbiaceae', 'Malvaceae', 'Myristicaceae', 'Elaeocarpaceae',
        'Annonaceae', 'Nyctaginaceae', 'Apocynaceae', 'Goupiaceae',
        'Ebenaceae', 'Meliaceae', 'Violaceae', 'Olacaceae',
        'Caryocaraceae', 'Combretaceae', 'Loganiaceae', 'Myrtaceae',
        'Simaroubaceae', 'Rubiaceae', 'Bignoniaceae', 'Malpighiaceae',
        'Salicaceae', 'Boraginaceae', 'Anacardiaceae', 'Lamiaceae',
        'Ochnaceae', 'Clusiaceae', 'Calophyllaceae'], dtype=object),
 40)

# Transformações


## Wood Density

In [145]:
wood_density_dataframe = wood_density_dataframe.rename(
    {
        "Family": "family.name",
        "Binomial": "scientific.name",
        "Wood density (g/cm^3), oven dry mass/fresh volume": "wood.density",
        "Reference Number": "reference.number",
    },
    axis=1,
)

In [146]:
mean_wood_density_dataframe = (
    wood_density_dataframe.groupby(["family.name", "scientific.name"])["wood.density"]
    .mean("wood.density")
    .reset_index()
)

In [147]:
wood_density_dataframe.head()

,Number,family.name,scientific.name,wood.density,Region,reference.number
0,1,Fabaceae,Abarema jupunba,0.78000,South America (tropical),114
1,2,Fabaceae,Abarema jupunba,0.66000,South America (tropical),198
2,3,Fabaceae,Abarema jupunba,0.55104,South America (tropical),52
3,4,Fabaceae,Abarema jupunba,0.53382,South America (tropical),65
4,5,Fabaceae,Abarema jupunba,0.55104,South America (tropical),189


In [148]:
inventory = inventory.merge(
    mean_wood_density_dataframe,
    on="scientific.name",
    how="left",
)

In [149]:
inventory = inventory.rename(
    {
        "family.name_x": "family.name",
        "Binomial": "scientific.name",
        "wood.density": "family.scientific.wood.density",
        "Reference Number": "reference.number",
    },
    axis=1,
)

In [150]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,family.scientific.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333


In [151]:
mean_wood_density_dataframe = (
    wood_density_dataframe.groupby(["family.name"])["wood.density"]
    .mean("wood.density")
    .reset_index()
)

In [152]:
mean_wood_density_dataframe = mean_wood_density_dataframe.rename(
    {
        "wood.density": "family.wood.density",
    },
    axis=1,
)

mean_wood_density_dataframe.head()

,family.name,family.wood.density
0,Acanthaceae,0.670709
1,Achariaceae,0.605300
2,Actinidiaceae,0.398541
3,Adoxaceae,0.446333
4,Akaniaceae,0.560710


In [153]:
inventory = inventory.merge(mean_wood_density_dataframe, on="family.name", how="left")


# wood_density_dataframe["reference.number"].apply(str)

In [154]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,family.scientific.wood.density,family.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775,0.771500
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953,0.344090
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403,0.702085
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333,0.678134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,20160830,840072.5767,9674390.969,Vochysiaceae,0.523047,0.551651
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,20160830,840071.0245,9674395.507,Fabaceae,0.585129,0.678134
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,20160830,840069.0191,9674396.896,Malvaceae,0.410000,0.476090
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,20160830,840069.1150,9674398.094,NaN,NaN,0.697848


In [155]:
inventory = inventory.drop("family.name_y", axis=1)

In [156]:
inventory = inventory.rename(
    {"family.name_x": "family.name", "scientific.name_x": "scientific.name"},
    axis=1,
)

In [157]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,0.771500
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,0.344090
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,0.702085
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,0.678134


In [158]:
inventory["wood.density"] = inventory["family.scientific.wood.density"].combine_first(
    inventory["family.wood.density"]
)

In [159]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,0.780701,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,0.771500,0.835775
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,0.344090,0.435953
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,0.702085,0.799403
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,0.678134,0.763333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,20160830,840072.5767,9674390.969,0.523047,0.551651,0.523047
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,20160830,840071.0245,9674395.507,0.585129,0.678134,0.585129
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,20160830,840069.0191,9674396.896,0.410000,0.476090,0.410000
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,20160830,840069.1150,9674398.094,NaN,0.697848,0.697848


In [160]:
mean_wood_density = inventory[inventory["wood.density"].notnull()][
    "wood.density"
].mean()

In [161]:
inventory = inventory.fillna(value={"wood.density": mean_wood_density})

## Tree height

In [162]:
inventory["tree.height"] = random.choice([38.1 + 1.8, 38.1 + 1.8]) * (
    1
    - np.exp(
        random.choice([-0.0693 + 0.0044, -0.0693 - 0.0044])
        * inventory["DBH"] ** random.choice([0.826 + 0.040, 0.826 - 0.040])
    )
)

In [163]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density,tree.height
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,0.780701,0.780701,35.328247
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,0.771500,0.835775,34.152400
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,0.344090,0.435953,30.654063
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,0.702085,0.799403,33.083757
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,0.678134,0.763333,16.679405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,20160830,840072.5767,9674390.969,0.523047,0.551651,0.523047,27.334167
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,20160830,840071.0245,9674395.507,0.585129,0.678134,0.585129,20.565010
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,20160830,840069.0191,9674396.896,0.410000,0.476090,0.410000,26.234924
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,20160830,840069.1150,9674398.094,NaN,0.697848,0.697848,18.635737


In [164]:
inventory = inventory.merge(
    inventory.groupby("plot")["tree.height"]
    .mean()
    .reset_index()
    .rename({"tree.height": "lorey.height"}, axis=1),
    on="plot",
)

## Snag density

In [165]:
inventory["snag.density"] = (
    (31 * 0.71)
    + (26 * 0.69)
    + (24 * 0.66)
    + (18 * 0.59)
    + (18 * 0.33)
    + (113 * 0.52)
    + (43 * 0.5)
    + (88 * 0.6)
    + (35 * 0.7)
    + (48 * 0.58)
    + (52 * 0.45)
    + (21 * 0.34)
    + (103 * 0.36)
    + (86 * 0.45)
) / (31 + 26 + 24 + 18 + 18 + 113 + 43 + 88 + 35 + 48 + 52 + 21 + 103 + 86)

In [166]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density,tree.height,lorey.height,snag.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,0.780701,0.780701,35.328247,27.01936,0.51568
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,0.771500,0.835775,34.152400,27.01936,0.51568
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,0.344090,0.435953,30.654063,27.01936,0.51568
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,0.702085,0.799403,33.083757,27.01936,0.51568
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,0.678134,0.763333,16.679405,27.01936,0.51568


## Tree vs Palms vs Lianas

In [167]:
inventory["palm"] = np.where(inventory["family.name"].str.contains("Arecaceae"), 1, 0)
inventory["palm"] = np.where(
    inventory["family.name"].str.contains("Palmae"), 1, inventory["palm"]
)
inventory["palm"] = np.where(
    inventory["family.name"].str.contains("Palmaceae"), 1, inventory["palm"]
)

inventory.palm.unique()

array([0, 1])

In [168]:
inventory["palm"] = np.where(
    (inventory["family.name"].str.contains("Arecaceae"))
    | (inventory["family.name"].str.contains("Palmae"))
    | (inventory["family.name"].str.contains("Palmaceae")),
    1,
    0,
)

In [169]:
inventory[inventory['palm'] == 1]["palm"].count()

55

In [170]:
inventory["liana"] = np.where(
    inventory["common.name"].str.contains("liana"),
    1,
    0,
)

In [171]:
inventory[inventory['liana'] == 1]["palm"].count()

7

In [172]:
inventory[
    (inventory["family.wood.density"].isna())
    & (inventory["family.scientific.wood.density"].isna())
]["common.name"].unique()

array(['morta', 'tamaguare', 'ucuuba da varzea', 'carana', 'fava rosa',
       'pincel de macaco', 'breu brrote', 'abacabeira', 'jarana vermelha',
       'taguari', 'tanenbuco amarelo', 'acaizeiro', 'liana',
       'faveira amargosa', 'zaguari', 'virola/ucuuba preta',
       'muiracatiara rajada', 'abiu rosadinha', 'burra leteira',
       'abirana', 'gaivota', 'abirana branca', 'carape', 'mirindiba',
       'pedra ume', 'brteu branco', 'timbora', 'mamominho',
       'ucuuba folha grande', 'tanenbuco', 'faveira rosa',
       'nao identificada', 'guajara vermelho', 'muiracacaco',
       'acariguara', 'acariguarana', 'puriu', ' liana', 'seringuarana',
       'nao indetificada', 'manguirana', 'virola(ucuuba)', 'sorvao',
       'urucurana branca', 'tento cachua', 'ipê amarelo', 'uxi coroa',
       'quariuba', 'mirapiranga', 'ucuuba t.f', 'envira cunacaru',
       'seringarana preta', 'pocoro', 'cacaui', 'urucurana branco'],
      dtype=object)

In [175]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,...,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density,tree.height,lorey.height,snag.density,palm,liana
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,...,840127.0705,9674180.998,NaN,0.780701,0.780701,35.328247,27.01936,0.51568,0,0
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,...,840098.9678,9674177.868,0.835775,0.771500,0.835775,34.152400,27.01936,0.51568,0,0
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,...,840126.4377,9674180.705,0.435953,0.344090,0.435953,30.654063,27.01936,0.51568,0,0
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,...,840109.4829,9674171.803,0.799403,0.702085,0.799403,33.083757,27.01936,0.51568,0,0
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,...,840125.6847,9674181.313,0.763333,0.678134,0.763333,16.679405,27.01936,0.51568,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,...,840072.5767,9674390.969,0.523047,0.551651,0.523047,27.334167,25.30618,0.51568,0,0
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,...,840071.0245,9674395.507,0.585129,0.678134,0.585129,20.565010,25.30618,0.51568,0,0
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,...,840069.0191,9674396.896,0.410000,0.476090,0.410000,26.234924,25.30618,0.51568,0,0
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,...,840069.1150,9674398.094,NaN,0.697848,0.697848,18.635737,25.30618,0.51568,0,0


## ACD

In [176]:
fc = 0.5

living_trees_eq = (
    0.0673
    * fc
    * (inventory["wood.density"] * (inventory["DBH"] ** 2) * inventory["tree.height"])
    ** 0.976
)

living_palms_eq = 0.03781 * fc * (inventory["DBH"] ** 2.7483)

living_lianas_eq = 0.03798 * fc * (inventory["DBH"] ** 2.657)

dead_trees_eq = (
    0.1007
    * fc
    * inventory["snag.density"]
    * (inventory["DBH"] ** 2)
    * (inventory["tree.height"] ** 0.818)
)

In [178]:
inventory["ACD"] = np.where(
    (inventory["palm"] == 0) & (inventory["liana"] == 0) & (inventory["Dead"] == False),
    living_trees_eq,
    np.where(
        (inventory["palm"] == 1)
        & (inventory["liana"] == 0)
        & (inventory["Dead"] == False),
        living_palms_eq,
        np.where(
            (inventory["palm"] == 0)
            & (inventory["liana"] == 1)
            & (inventory["Dead"] == False),
            living_lianas_eq,
            dead_trees_eq,
        ),
    ),
)

In [179]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,...,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density,tree.height,lorey.height,snag.density,palm,liana,ACD
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,...,9674180.998,NaN,0.780701,0.780701,35.328247,27.01936,0.51568,0,0,1748.227009
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,...,9674177.868,0.835775,0.771500,0.835775,34.152400,27.01936,0.51568,0,0,1405.533412
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,...,9674180.705,0.435953,0.344090,0.435953,30.654063,27.01936,0.51568,0,0,355.298059
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,...,9674171.803,0.799403,0.702085,0.799403,33.083757,27.01936,0.51568,0,0,1060.041547
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,...,9674181.313,0.763333,0.678134,0.763333,16.679405,27.01936,0.51568,0,0,36.087963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,...,9674390.969,0.523047,0.551651,0.523047,27.334167,25.30618,0.51568,0,0,223.191616
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,...,9674395.507,0.585129,0.678134,0.585129,20.565010,25.30618,0.51568,0,0,65.868550
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,...,9674396.896,0.410000,0.476090,0.410000,26.234924,25.30618,0.51568,0,0,142.658575
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,...,9674398.094,NaN,0.697848,0.697848,18.635737,25.30618,0.51568,0,0,51.739447
